# Add time series to `System`

**Originally Contributed by**: Clayton Barrows

## Introduction

An example of how to parse add time series data to a `System` using [PowerSystems.jl](github.com/NREL-SIIP/PowerSystems.jl)

For example, a `System` created by [parsing a MATPOWER file](../../notebook/2_PowerSystems_examples/parse_matpower.ipynb)
doesn't contain any time series data. So a user may want to add time series to the `System`
### Dependencies
Let's use the 5-bus dataset we parsed in the MATPOWER example

In [1]:
using SIIPExamples
using PowerSystems
import JSON3
using Logging
logger = configure_logging(console_level = Logging.Error, file_level = Logging.Info, filename = "ex.log")
pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(joinpath(pkgpath, "test", "2_PowerSystems_examples", "parse_matpower.jl"))

┌ Info: Precompiling SIIPExamples [2c79006f-6450-48c4-b124-fbadab4f299d]
└ @ Base loading.jl:1278


Project SIIPExamples v0.0.2
Status `~/sandboxes/SIIPExamples.jl/Project.toml`
  [336ed68f] CSV v0.8.0
  [9961bab8] Cbc v0.7.1
  [41994980] D3TypeTrees v0.1.1
  [a93c6f00] DataFrames v0.22.1
  [0b91fe84] DisplayAs v0.1.2
  [2cd47ed4] InfrastructureSystems v1.0.4
  [b6b21f68] Ipopt v0.6.3
  [0f8b85d8] JSON3 v1.4.0
  [98b081ad] Literate v2.7.0
  [f0f68f2c] PlotlyJS v0.14.0
  [91a5bcdd] Plots v1.9.1
  [5f7eddb3] PowerGraphics v0.6.1
  [e690365d] PowerSimulations v0.8.0 `https://github.com/NREL-SIIP/PowerSimulations.jl.git#master`
  [398b2ede] PowerSimulationsDynamics v0.3.0 `https://github.com/NREL-SIIP/PowerSimulationsDynamics.jl.git#master`
  [bcd98974] PowerSystems v1.0.2
  [c3572dad] Sundials v4.3.0
  [9e3dc215] TimeSeries v0.19.1
  [f269a46b] TimeZones v1.5.1
  [0f1e0344] WebIO v0.8.15
  [ade2ca70] Dates
  [56ddb016] Logging
  [44cfe95a] Pkg
  [9a3f8284] Random
  [10745b16] Statistics
  [8dfed614] Test


,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Arc,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,6
2,Area,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
3,Bus,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5
4,Line,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5
5,LoadZone,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
6,PhaseShiftingTransformer,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,2
7,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,3
8,RenewableDispatch,RenewableGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,2
9,ThermalStandard,ThermalGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5


### Define pointers to time series files
For example, if we want to add a bunch of time series files, say one for each load and
one for each renewable generator, we need to define pointers to each .csv file containing
the time series in the following format (PowerSystems.jl also supports a CSV format for this file)

In [5]:
FORECASTS_DIR = joinpath(base_dir, "forecasts", "5bus_ts")
fname = joinpath(FORECASTS_DIR, "timeseries_pointers_da.json")

open(fname, "r") do f
    JSON3.@pretty JSON3.read(f)
end

[
  {
                       "component_name": "SolarBusC",
                 "normalization_factor": 1,
                                 "name": "max_active_power",
     "scaling_factor_multiplier_module": "PowerSystems",
                            "data_file": "./gen/Renewable/PV/da_solar5.csv",
                           "resolution": 3600,
                               "module": "InfrastructureSystems",
                             "category": "Generator",
            "scaling_factor_multiplier": "get_max_active_power",
                           "simulation": "DAY_AHEAD",
                                 "type": "SingleTimeSeries"
  },
  {
                       "component_name": "WindBusA",
                 "normalization_factor": 1,
                                 "name": "max_active_power",
     "scaling_factor_multiplier_module": "PowerSystems",
                            "data_file": "./gen/Renewable/WIND/da_wind5.csv",
                           "resolution": 3600,
      

### Read the pointers

In [ ]:
ts_pointers = PowerSystems.IS.read_time_series_file_metadata(fname)

### Read and assign time series to `System` using the `ts_pointers` struct

In [6]:
add_time_series!(sys, fname)
sys

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Arc,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,6
2,Area,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
3,Bus,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5
4,Line,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5
5,LoadZone,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
6,PhaseShiftingTransformer,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,2
7,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,3
8,RenewableDispatch,RenewableGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,2
9,ThermalStandard,ThermalGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*